# Lending Club Case Study

In [157]:
# Importing core libraries required for the case study
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sea
import datetime as dt

debug = True
# Utility function to take a snapshot of the csv locally just to validate the outputs
def snapshot_data(df, snapshot_name): 
    if debug == True:
        print(df.shape)
        df.to_csv('./.data/snapshot.'+ snapshot_name +'.loan.csv')  

# Setting max rows settings to 200 to display all the summary data
pd.set_option("display.max_rows", 50)

## Loading Data

In [158]:
# Loading the complete dataset into variable df
df_loan = pd.read_csv('./.data/loan.csv', low_memory=False)

In [159]:
# Print summary of Nulls, Blanks in the dataset
if debug == True:
    print(df_loan.isnull().sum())

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64


## Step1 - Dropping Rows - where loan_status = "Current"

In [160]:
# The rows where loan_stats=Current are the data where the loan repayment is currently in progress
# The loans which are currently in progress will not contribute to decisions 
# of default or pass as it's difficult to predict the outcome
#
# Dropping the rwos early as, dropping all Currrent rows introduces NA columns which can be easily dropped
rows_before = len(df_loan)
df_clean = df_loan[df_loan['loan_status'] != "Current"]

In [161]:
# Print current data statistics after dropping rows with loan_status "CURRENT"
rows_after = len(df_clean)
if debug == True:
    print("Number of rows dropped = ", (rows_before - rows_after))
    print("Percentage of rows dropped = ", round((rows_before - rows_after)/rows_before*100,2),"%")

Number of rows dropped =  1140
Percentage of rows dropped =  2.87 %


## Step2 - Dropping Columns

In [162]:
# Print the initial shape of the array before dropping columns
if debug == True:
    print(df_clean.shape)

(38577, 111)


In [163]:
# Dropping columns which is unique id in nature. They dont contribute to loan analysis
df_clean = df_clean.drop(['id','member_id'],  axis=1)

# Dropping text/description columns which wont contribute to overall analysis
# These are names of establishment etc which will not contribute to loan pass or failure
# THe URL column is a static link with id as the attribute. Its a redundant column
df_clean = df_clean.drop(['url', 'emp_title', 'desc', 'title'],  axis=1)

# Dropping column sub_grade as the current analysis will limit to Grade only
df_clean = df_clean.drop(['sub_grade'],  axis=1)

In [164]:
# Dropping all columns which refer to behavoural data of customer post loan approval 
# Behaviour data of the customers are captured post the loan approval
# The data is not available at the time of loan approval and thus cannot be used for calculations
df_clean = df_clean.drop(['delinq_2yrs', 'earliest_cr_line', 
                          'inq_last_6mths', 'open_acc', 'pub_rec', 
                          'revol_bal', 'revol_util', 'total_acc', 
                          'out_prncp', 'out_prncp_inv', 'total_pymnt', 
                          'total_pymnt_inv', 'total_rec_prncp', 
                          'total_rec_int', 'total_rec_late_fee', 'recoveries', 
                          'collection_recovery_fee', 'last_pymnt_d', 
                          'last_pymnt_amnt', 'last_credit_pull_d', 
                          'application_type'],  axis=1)

In [165]:
# Print the shape of dataframe post dropping behaviour, redundant and other columns which dont contribute to analysis
if debug == True:
    print(df_clean.shape)

(38577, 83)


In [166]:
# Dropping all columns whose all the values are NA
# Print all NA columns for verification
if debug == True:
    print("Columns with all values as NA", df_clean.columns[df_clean.isna().all()].tolist())

# Dropping all the columns whose all the records are NaN or Null
df_clean = df_clean.dropna(axis='columns', how="all")

Columns with all values as NA ['next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_m

In [167]:
# Print the shape of dataframe post dropping behaviour, redundant and other columns which dont contribute to analysis
if debug == True:
    print(df_clean.shape)

(38577, 28)


In [168]:
# Dropping all columns with all zero values
df_clean = df_clean.loc[:, (df_clean != 0).any(axis=0)]

In [169]:
# Function to Drop all columns who have constant values (ignoring NA value)
# Example most of the columns is 1 and rest is NA, the column will be dropped
# If we have 1,2 and NA, the column wont be dropped
def drop_constant_columns(df):
    for c in df.columns:
        if df[c].nunique(dropna=True) == 1:
            if debug == True:
                print(c)
            df = df.drop(c, axis=1)
    return df

# Drop all constant columns from df1 (definition of constant is constant value across the rows, this ignores Na values)
df_clean = drop_constant_columns(df_clean)

pymnt_plan
initial_list_status
collections_12_mths_ex_med
policy_code
chargeoff_within_12_mths
tax_liens


In [170]:
# Print the shape of dataframe post dropping columns having constant values. 
# This includes columns which has constant + NA values as well
if debug == True:
    print(df_clean.shape)

(38577, 20)


In [171]:
# Function which checks the amount of empty values in a dataframe and 
# drops the column if the amount of empty values is more than 65%
# 60% is the threshhold percentage which decides imputing vs dropping 
def drop_mostly_empty_columns(df):
    total_rows = len(df)
    for c in df.columns:
        # Drop columns whose mean na values exceed 65%
        if df[c].isna().mean().round(2) >= 0.65:
            if debug == True:
                print(c)
            df = df.drop(c, axis=1)
    return df
df_clean = drop_mostly_empty_columns(df_clean)

mths_since_last_delinq
mths_since_last_record


In [172]:
# Print the shape of the dataframe after dropping columns whose values are empty = more than 65%
if debug == True:
    print(df_clean.shape)

(38577, 18)


## Step3 - Convert the data types

In [173]:
# Convert the columns loan_amnt and funded_amnt as flot64
df_clean = df_clean.astype({'loan_amnt':'float','funded_amnt':'float'})

In [174]:
# Convert the term column into an integer from a string
df_clean['term'] = df_clean['term'].apply(lambda x : int(x[:-7]))

In [175]:
# Convert int_rate to  float by removing the "%" character
df_clean['int_rate'] = df_clean['int_rate'].apply(lambda x : float(x[:-1]))

In [176]:
# Converting the loan_status to boolean column. "Fully-Paid is True and Charged Off is False"
# Added a function instead of lambda because, if this is accidentally re-run on a boolean column, the logic broke
# Now it will only convert to boolean if the column is a string and has the two specific values
def convert_loan_status_to_boolean(x):
    if x == "Fully Paid":
        return True
    elif x == "Charged Off":
        return False
    else:
        return x

df_clean['loan_status'] = df_clean['loan_status'].apply(lambda x: convert_loan_status_to_boolean(x))

In [177]:
# Converting the column issue_d from string object to DateTime
df_clean['issue_d'] = pd.to_datetime(df_clean['issue_d'], format='%b-%y')

# Adding additional column for Year and Month for analysis extrating Year and Month from issue_d
df_clean['issue_y'] = pd.DatetimeIndex(df_clean['issue_d']).year
df_clean['issue_m'] = pd.DatetimeIndex(df_clean['issue_d']).month

In [178]:
# Print the shape of the dataframe after reformatting the columns datatypes
if debug == True:
    print(df_clean.shape)

(38577, 20)


## Step 4 - Identify columns with blank values which need to be imputed

In [179]:
# Identify columns who have blank values and what percentage of total values are there blanks. 
# These values may need to be imputed
for c in df_clean.columns[df_clean.isna().any()].tolist():
    print(c, round(len(df_clean[df_clean[c].isna()]) / len(df_clean) * 100,2),"%")

emp_length 2.68 %
pub_rec_bankruptcies 1.81 %


Here two kinds of decision can be taken
* **Option 1** - Drop the rows with blank values
* **Option 2** - Impute the values with a median value

In [180]:
# Print the current dimensions of the dataframe
rows_before = len(df_clean)
print(df_clean.shape)

(38577, 20)


In [181]:
# Since the percent of rows is very small, dropping the rows instead of imputing them
df_clean = df_clean[df_clean['emp_length'].notna()]
df_clean = df_clean[df_clean['pub_rec_bankruptcies'].notna()]

In [182]:
# Print the dimensions of the dataframe after dropping rows
rows_after = len(df_clean)
print(df_clean.shape)
print("Number of rows dropped = ,", (rows_before - rows_after))
print("Percentage of rows dropped = ", round((rows_before - rows_after)/rows_before*100,2),"%")

(36847, 20)
Number of rows dropped = , 1730
Percentage of rows dropped =  4.48 %


In [192]:
# Converting emp_length to a string object explicitly as python id giving intermitent error of float
df_clean['emp_length'] = df_clean['emp_length'].apply(lambda x: str(x))
# Remove years keyword from the emp_lemgth column
# Remove + and < characters
# Consider 10 + as 10 and <1 as 1 for the sake of this analysis
# This is because anything 10+ is represented as 10 + years, there is no distinct and unique value available
# So replacing 10+ to 10 will not impact to the resolution of the data
# Similarly replacing <1 to 1 will not impact the analysis as well
df_clean['emp_length'] = df_clean['emp_length'].apply(lambda x: int(x.rstrip("year").rstrip("years").rstrip(" ").rstrip("+").lstrip("<").lstrip()))
# Will add a column later to bin the data

In [193]:
# We will also try to use median to impute the values and compare both outcomes

In [194]:
# Printing column info to analyse missing values, empty values in a column
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36847 entries, 0 to 39680
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   loan_amnt             36847 non-null  float64       
 1   funded_amnt           36847 non-null  float64       
 2   funded_amnt_inv       36847 non-null  float64       
 3   term                  36847 non-null  int64         
 4   int_rate              36847 non-null  float64       
 5   installment           36847 non-null  float64       
 6   grade                 36847 non-null  object        
 7   emp_length            36847 non-null  int64         
 8   home_ownership        36847 non-null  object        
 9   annual_inc            36847 non-null  float64       
 10  verification_status   36847 non-null  object        
 11  issue_d               36847 non-null  datetime64[ns]
 12  loan_status           36847 non-null  bool          
 13  purpose         

In [195]:
# Always take one final snapshot - Temp file
df_clean.to_csv('./.data/snapshot.clean.loan.csv') 